In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import *  
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
from xgboost import *
from sklearn.metrics import classification_report,confusion_matrix,precision_score,accuracy_score,recall_score,f1_score

In [2]:
df = pd.read_csv(r'C:\Users\USER1\Desktop\Data_for_UCI_named.csv')

In [3]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stab', 'stabf'],
      dtype='object')

In [5]:
df.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [6]:
print(df['stabf'].value_counts())

unstable    6380
stable      3620
Name: stabf, dtype: int64


In [7]:
minority_length, minority_width = df[df['stabf'] == "unstable"].shape
majority_length, majority_width = df[df['stabf'] == "stable"].shape

minority_percent = round(100*minority_length/(minority_length+majority_length),2)
minority_percent

63.8

In [8]:
df["stabf"].replace({"stable":1, "unstable":0}, inplace= True)

In [9]:
x= df.drop(columns='stabf')
y=df['stabf']

In [10]:
x.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860


In [11]:
y.head()

0    0
1    1
2    0
3    0
4    0
Name: stabf, dtype: int64

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state= 42)

In [13]:
train_data = x_train.join(y_train, on=x_train.index)
train_data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
9254,6.587242,8.255999,8.109960,4.457904,4.332937,-1.073779,-1.371388,-1.887770,0.166933,0.612514,0.792247,0.836127,0.063720,0
1561,0.676193,0.781086,5.556240,3.765235,2.931210,-1.062958,-1.175796,-0.692456,0.740131,0.772031,0.420697,0.839775,-0.047254,1
1670,4.757109,8.891698,1.778212,9.795918,5.238057,-1.961965,-1.705948,-1.570144,0.850271,0.601201,0.302983,0.126409,0.014128,0
6087,3.343688,8.461652,8.675216,2.197498,3.360289,-1.189211,-1.064192,-1.106886,0.961836,0.686392,0.444109,0.696517,0.024980,0
6669,7.581260,4.061263,6.192641,6.557269,3.221193,-1.433562,-0.591071,-1.196560,0.217076,0.680618,0.457295,0.726299,0.050475,0


In [14]:
test_data = x_test.join(y_test, on=x_test.index)
test_data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
6252,1.953136,9.692422,6.570391,7.737644,5.128952,-1.860248,-1.380687,-1.888018,0.487235,0.934229,0.433606,0.328522,0.035629,0
4684,4.613690,9.423079,1.424423,7.372338,4.655631,-1.716394,-1.182286,-1.756951,0.395106,0.772211,0.911199,0.328193,0.002383,0
1731,2.738001,1.341172,7.986504,9.842747,5.007610,-1.275363,-1.877241,-1.855006,0.162310,0.175994,0.897505,0.601356,0.030620,0
4742,2.006512,2.311770,4.410704,2.726592,4.396237,-1.532751,-1.327943,-1.535543,0.053894,0.573440,0.061032,0.775629,-0.021962,1
4521,3.993497,3.742481,8.975171,1.182813,2.875057,-1.170108,-1.117862,-0.587086,0.100005,0.753533,0.437366,0.611471,-0.039032,1


In [15]:
X = train_data.drop(columns=["stabf"])
Y = train_data["stabf"]

In [16]:
smote = SMOTE(sampling_strategy="auto")
x, y = smote.fit_resample(X, Y)
y.value_counts()

0    5073
1    5073
Name: stabf, dtype: int64

In [17]:
# Splitting the dataframe
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state= 42,)
y_train.value_counts()

0    4078
1    4038
Name: stabf, dtype: int64

In [18]:
rf= RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [19]:
pred = rf.predict(X_test)

In [20]:
pred

array([0, 0, 1, ..., 0, 0, 1], dtype=int64)

In [21]:
accuracy_score(y_test,pred)

1.0

In [25]:
lgb = lgb.LGBMClassifier(random_state=42)
lgb.fit(X_train, y_train)

AttributeError: 'LGBMClassifier' object has no attribute 'LGBMClassifier'

In [27]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [28]:
pred = xgb.predict(X_test)

In [29]:
accuracy_score(y_test,pred)

1.0